In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import datetime

In [ ]:
# --- Configura o Selenium para rodar em modo "headless" (sem abrir janela)
chrome_options = Options()
chrome_options.add_argument("--headless")  # Remova essa linha se quiser ver o navegador
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# --- Inicializa o navegador
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
url_qrcode = "http://nfe.sefaz.ba.gov.br/servicos/nfce/modulos/geral/NFCEC_consulta_chave_acesso.aspx?p=29250306057223048450650150000775861151017568|2|1|1|8DF25CB77517F73F80677AE283AEE7DFCCA38282"
driver.get(url_qrcode)

In [ ]:
# --- Aguarda a página carregar completamente
time.sleep(5)  # Pode ajustar dependendo da conexão

# --- Extrai o HTML renderizado
html = driver.page_source
driver.quit()

In [ ]:
# --- Faz o parsing com BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

In [ ]:
#  use the method prettify() to display the HTML in the nested structure
#print(soup.prettify())

#### Tags
The Tag object corresponds to an HTML tag in the original document

In [ ]:
# Print the Title Tag
tag_object = soup.title
print(tag_object)

#### HTML Attributes


In [ ]:
#
tag_object = soup.p
print(tag_object.string)

### Filter

##### find_all
The `find_all()` method looks through a tag's descendants and retrieves all descendants that match your filters.

The Method signature for `find_all(name, attrs, recursive, string, limit, **kwargs)`

In [ ]:
table_rows = soup.find_all('tr')
table_rows

In [ ]:
# Get the Suplier Name
supplier_name = soup.find(id="u20").string
print(supplier_name, len(supplier_name))

In [ ]:
# Get the CNPJ Number and Address

# --- Pega todos os <div class="text">
divs_text = soup.find_all('div', class_='text')
#print(divs_text[1].text) # pode usar o .text para obter os valores

# Inicializa valores
cnpj = ""
endereco = ""

for i, div in enumerate(divs_text):
    texto = div.text.strip()
    #print(i, text, len(text))
    if 'CNPJ' in texto:
        # Extrai CNPJ com regex
        cnpj_match = re.search(r"\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}", texto)

        if cnpj_match:
            cnpj = cnpj_match.group()
        
        # Tenta pegar o próximo div como endereço
        if i + 1 < len(divs_text):
            endereco = divs_text[i + 1].text.strip()
        break


print("CNPJ:", cnpj)
print("Endereço:", endereco)



In [ ]:
""" Obter as seguintes informações
1. Informações Gerais da Nota
  a. Numero = 77586
  b. Emissão =  08/03/2025 21:46:15-03:00


"""


In [ ]:
infos = soup.find_all('li')
print(infos[0])

In [ ]:
# Get the Suplier Name
infos = soup.find_all('div', class_="ui-li-static ui-body-inherit ui-first-child ui-last-child")
print(infos)

In [ ]:
# --- Localiza o <li> que contém "Número:"
li_info = soup.find("li")
print(li_info)

In [ ]:
# --- Extrai o texto bruto
info_text = li_info.get_text(" ", strip=True)
print(info_text)

In [ ]:
# --- Regex para capturar os valores
numero = re.search(r"Número:\s*(\d+)", info_text).group(1)
serie = re.search(r"Série:\s*(\d+)", info_text).group(1)
emissao = re.search(r"Emissão:\s*([\d/ :\-]+)", info_text)

emissao_val = emissao.group(1).strip() if emissao else ""

num_nfe = numero + "-" + serie
print(emissao_val)

#### teste 2 - devido não ter retornado a data de emissão corretamente

In [ ]:
# --- Localiza o <li> com as informações
li_info = soup.find("li")

# Dicionário para armazenar campos encontrados
dados = {}


In [ ]:
dados = {}
for strong in li_info.find_all("strong"):
    label = strong.text.strip().replace(":", "")
    next_text = strong.next_sibling
    if next_text:
        value = next_text.strip()
        dados[label] = value

# --- Extrai os valores
numero_val = dados.get("Número", "")
serie_val = dados.get("Série", "")
emissao_val = dados.get("Emissão", "")

# --- Remove qualquer coisa após " -", mantendo apenas a data/hora com fuso
emissao_limpo = emissao_val.split(" -")[0].strip()

# Converte para datetime
emissao_datetime = datetime.strptime(emissao_limpo, "%d/%m/%Y %H:%M:%S%z")

# --- Exibe resultados
print("Número:", numero_val)
print("Série:", serie_val)
print("Emissão (datetime):", emissao_datetime)

O método .next_sibling retorna o próximo “irmão” (ou seja, o próximo nó do mesmo nível) que vem depois do elemento atual no HTML.

Em HTML, os elementos "irmãos" são os que estão dentro do mesmo pai (parent).

In [ ]:
li_info2 = soup.find("li")

for strong in li_info2.find_all("strong"):
    label = strong.text.strip().replace(":", "")
    next_text0 = strong.text.strip()
    next_text1 = strong.next_sibling
    print(next_text0, next_text1) 